## 2. Analyse statistiques bivariées

Ce notebook suit les étapes suivantes, en cohérence avec les consignes de `AGENTS.md` :
1. Charger les données depuis le dossier `data/`.
2. Examiner les tableaux de contingence pour détecter les liens entre variables.
3. Tester la significativité statistique avec le Chi².
4. Mesurer la force des associations à l'aide du V de Cramér.


### a. Tableau de contingence

Les tableaux sont tronqués avec `head()` pour conserver une lecture fluide ; le nombre de modalités masquées est indiqué.


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency  # type: ignore


In [2]:
from pathlib import Path


def find_project_root(start: Path) -> Path:
    """Climb up until a directory containing 'data' exists (max 6 levels)."""
    current = start
    for _ in range(6):
        if (current / 'data').exists():
            return current
        if current.parent == current:
            break
        current = current.parent
    return start

PROJECT_ROOT = find_project_root(Path.cwd())
file_path = PROJECT_ROOT / 'data' / 'dataset.csv'
print(f"Chemin du fichier détecté: {file_path}")

df = pd.read_csv(file_path)


Chemin du fichier détecté: /Users/steven/Documents/Programmation/MentalHealth_CatBoost/data/dataset.csv


In [3]:
def tableau_contingence(df, target='Treatment', max_modalites=8):
    """Affiche un aperçu des tableaux de contingence avec la cible."""
    print('=' * 80 + f"\nTABLEAUX DE CONTINGENCE - TOUTES LES VARIABLES × {target}\n" + '=' * 80)
    resultats = {}
    for col in df.columns:
        if col == target:
            continue
        print(f"\n{'=' * 80}\nVariable : {col} × {target}\n{'=' * 80}")
        contingence = pd.crosstab(df[col], df[target], margins=True, margins_name='TOTAL')
        apercu_effectifs = contingence.head(max_modalites)
        print("\n--- EFFECTIFS (aperçu) ---")
        print(apercu_effectifs)
        if contingence.shape[0] > max_modalites:
            print(f"... ({contingence.shape[0] - max_modalites} modalités supplémentaires masquées)")
        contingence_row = pd.crosstab(df[col], df[target], normalize='index') * 100
        apercu_pourcentage = contingence_row.round(2).head(max_modalites)
        print(f"\n--- PROPORTIONS EN LIGNE (%) - Distribution de {target} selon {col} (aperçu) ---")
        print(apercu_pourcentage)
        if contingence_row.shape[0] > max_modalites:
            print(f"... ({contingence_row.shape[0] - max_modalites} modalités supplémentaires masquées)")
        resultats[col] = {'effectifs': contingence, 'pct_ligne': contingence_row}
    print("\n" + "=" * 80 + "\nANALYSE TERMINÉE\n" + "=" * 80)
    return resultats


# Analyse de toutes les variables avec Treatment
resultats_contingence = tableau_contingence(df)


TABLEAUX DE CONTINGENCE - TOUTES LES VARIABLES × Treatment

Variable : Gender × Treatment

--- EFFECTIFS (aperçu) ---
Treatment      No     Yes   TOTAL
Gender                           
Female       9406   21308   30714
Male       123552  107062  230614
TOTAL      132958  128370  261328

--- PROPORTIONS EN LIGNE (%) - Distribution de Treatment selon Gender (aperçu) ---
Treatment     No    Yes
Gender                 
Female     30.62  69.38
Male       53.58  46.42

Variable : Country × Treatment

--- EFFECTIFS (aperçu) ---
Treatment                 No   Yes  TOTAL
Country                                  
Australia               1397  2147   3544
Belgium                  477     0    477
Bosnia and Herzegovina   237     0    237
Brazil                   886   452   1338
Canada                  6802  7375  14177
Colombia                 384     0    384
Costa Rica               384     0    384
Croatia                    0   385    385
... (28 modalités supplémentaires masquées)

--- PRO

- Les femmes auraient une probabilité plus élevée d'être sous traitement que les hommes.
- Avoir un antécédent familial augmenterait la probabilité de traitement.
- Bénéficier d'une assurance santé améliorerait les chances d'être traité.
Faits intéressants et contre-intuitifs :
    - Sortir souvent ou non n'altère pas la probabilité d'être pris en charge.
    - Avoir des antécédents personnels de santé mentale ne change pas notablement les probabilités.
    - Les variations d'humeur n'entraînent pas forcément une hausse de la probabilité de traitement.
    - Les difficultés à gérer les émotions n'augmentent pas la probabilité d'être traité.


### b. Chi² et p-value

Les résultats sont triés par Chi² décroissant et seuls les 15 premiers sont affichés pour préserver la lisibilité.


In [4]:
def test_chi2(df, target='Treatment', max_variables=15):
    """Calcule Chi² et p-value pour chaque variable par rapport à la cible."""
    print('=' * 80 + f"\nTEST DU CHI² - ASSOCIATION AVEC {target}\n" + '=' * 80)
    resultats = []
    for col in df.columns:
        if col == target:
            continue
        contingence = pd.crosstab(df[col], df[target])
        chi2, p_value, _, _ = chi2_contingency(contingence)
        chi2 = float(chi2)  # type: ignore
        p_value = float(p_value)  # type: ignore
        p_str = '< 0.001' if p_value < 0.001 else round(p_value, 4)
        resultats.append({'Variable': col, 'Chi²': round(chi2, 2), 'p-value': p_str})
    resultats_df = pd.DataFrame(resultats).sort_values('Chi²', ascending=False)
    apercu = resultats_df.head(max_variables)
    print("\n" + apercu.to_string(index=False))
    if len(resultats_df) > max_variables:
        print(f"... ({len(resultats_df) - max_variables} variables supplémentaires masquées)")
    print("\n" + "=" * 80)
    print("Interprétation : p-value < 0.05 = association statistiquement significative")
    print("=" * 80 + "\nANALYSE TERMINÉE\n" + "=" * 80)
    return resultats_df


# Calcul pour toutes les variables
resultats_chi2 = test_chi2(df)


TEST DU CHI² - ASSOCIATION AVEC Treatment

             Variable     Chi² p-value
        FamilyHistory 34235.86 < 0.001
          CareOptions 22323.95 < 0.001
              Country 17572.25 < 0.001
               Gender  5711.56 < 0.001
MentalHealthInterview  2194.07 < 0.001
         SelfEmployed   362.10 < 0.001
     IncreasingStress    15.94 < 0.001
      CopingStruggles    11.46 < 0.001
           Occupation     9.49  0.0499
  MentalHealthHistory     5.51  0.0637
          DaysIndoors     3.74  0.4423
           MoodSwings     1.62   0.444
         WorkInterest     1.49  0.4749
       SocialWeakness     1.02     0.6
     SocialWeakness.1     1.02     0.6
... (1 variables supplémentaires masquées)

Interprétation : p-value < 0.05 = association statistiquement significative
ANALYSE TERMINÉE


### c. Cramér's V

Nous affichons les 15 associations les plus fortes afin de nous concentrer sur les signaux majeurs.


In [5]:
def cramer_v(df, target='Treatment', max_variables=15):
    """Calcule le V de Cramér pour mesurer la force d'association entre les variables."""
    print('=' * 80 + f"\nCRAMÉR'S V - FORCE D'ASSOCIATION AVEC {target}\n" + '=' * 80)
    resultats = []
    for col in df.columns:
        if col == target:
            continue
        contingence = pd.crosstab(df[col], df[target])
        chi2, _, _, _ = chi2_contingency(contingence)
        n = contingence.sum().sum()
        r, c = contingence.shape
        v = np.sqrt(chi2 / (n * (min(r, c) - 1)))
        if v < 0.1:
            force = 'Négligeable'
        elif v < 0.3:
            force = 'Faible'
        elif v < 0.5:
            force = 'Modéré'
        else:
            force = 'Fort'
        resultats.append({'Variable': col, "Cramér's V": round(v, 3), 'Force': force})
    resultats_df = pd.DataFrame(resultats).sort_values("Cramér's V", ascending=False)
    apercu = resultats_df.head(max_variables)
    print("\n" + apercu.to_string(index=False))
    if len(resultats_df) > max_variables:
        print(f"... ({len(resultats_df) - max_variables} variables supplémentaires masquées)")
    print("\n" + "=" * 80)
    print("Interprétation : 0-0.1 (négligeable), 0.1-0.3 (faible), 0.3-0.5 (modéré), >0.5 (fort)")
    print("=" * 80 + "\nANALYSE TERMINÉE\n" + "=" * 80)
    return resultats_df


# Calcul pour toutes les variables
resultats_cramer = cramer_v(df)


CRAMÉR'S V - FORCE D'ASSOCIATION AVEC Treatment

             Variable  Cramér's V       Force
        FamilyHistory       0.362      Modéré
          CareOptions       0.292      Faible
              Country       0.259      Faible
               Gender       0.148      Faible
MentalHealthInterview       0.092 Négligeable
         SelfEmployed       0.038 Négligeable
     IncreasingStress       0.008 Négligeable
      CopingStruggles       0.007 Négligeable
           Occupation       0.006 Négligeable
  MentalHealthHistory       0.005 Négligeable
          DaysIndoors       0.004 Négligeable
         HabitsChange       0.002 Négligeable
           MoodSwings       0.002 Négligeable
       SocialWeakness       0.002 Négligeable
         WorkInterest       0.002 Négligeable
... (1 variables supplémentaires masquées)

Interprétation : 0-0.1 (négligeable), 0.1-0.3 (faible), 0.3-0.5 (modéré), >0.5 (fort)
ANALYSE TERMINÉE


La conclusion de l'analyse descriptive est que le jeu de données pourrait être synthétique ou biaisé car certains résultats sont contre-intuitifs, notamment pour l'historique de santé mentale. Les réponses étant auto-déclarées, une partie des biais peut provenir de là. L'objectif avec CatBoost sera d'observer les écarts entre le V de Cramér et les valeurs de SHAP : les relations non linéaires capturées par CatBoost pourraient mettre en avant des variables prédictives.
